In [18]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)
#         if batch_idx % log_interval == 0:
    
    print('\Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
    loss, correct, len(train_loader.dataset),
    100. * correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    # Training settings
    batch_size = 100
    test_batch_size = 100
    epochs = 10
    lr = 0.01
    momentum = 0.5
    no_cuda = False
    seed = 123
    log_interval = 100
    out_dir = './result'
    test_interval = 1
    resume_interval = 1
    
    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch,log_interval)
        test(model, device, test_loader)

In [19]:
main()

\Train set: Average loss: 0.6751, Accuracy: 37176/60000 (62%)

Test set: Average loss: 0.2601, Accuracy: 9301/10000 (93%)

\Train set: Average loss: 0.3413, Accuracy: 51572/60000 (86%)

Test set: Average loss: 0.1517, Accuracy: 9545/10000 (95%)

\Train set: Average loss: 0.2709, Accuracy: 53708/60000 (90%)

Test set: Average loss: 0.1153, Accuracy: 9656/10000 (97%)

\Train set: Average loss: 0.2136, Accuracy: 54754/60000 (91%)

Test set: Average loss: 0.0983, Accuracy: 9697/10000 (97%)

\Train set: Average loss: 0.2285, Accuracy: 55300/60000 (92%)

Test set: Average loss: 0.0904, Accuracy: 9731/10000 (97%)

\Train set: Average loss: 0.2297, Accuracy: 55679/60000 (93%)

Test set: Average loss: 0.0799, Accuracy: 9761/10000 (98%)

\Train set: Average loss: 0.1578, Accuracy: 55964/60000 (93%)

Test set: Average loss: 0.0757, Accuracy: 9759/10000 (98%)

\Train set: Average loss: 0.1658, Accuracy: 56232/60000 (94%)

Test set: Average loss: 0.0696, Accuracy: 9788/10000 (98%)

\Train set: Aver